In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import csv
import re

In [ ]:
def clear_numeric_fields(file: str):
    r = csv.reader(open(file=file))
    lines = list(r)
    new_lines = []

    for line in lines:
        new_lines.append([re.sub(r'_$', '', el) for el in line])

    with open(file=file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerows(new_lines)


In [ ]:
def input_pipeline(path: str) -> pd.DataFrame:

    #clear_numeric_fields(path)

    column_types = {"ID": str, #0
                    "Customer_ID": str, #0
                    "Month": str, #1
                    "Name": str, #2
                    "Age": np.int16, #3
                    "SSN": str, #4
                    "Occupation": str, #5
                    "Annual_Income": np.float32, #6
                    "Monthly_Inhand_Salary": np.float32, #7
                    "Num_Bank_Accounts": np.int16, #8
                    "Num_Credit_Card": np.int16, #9
                    "Interest_Rate": np.float32, #10
                    "Num_of_Loan": np.int16, #11
                    "Type_of_Loan": str, #12
                    "Delay_from_due_date": np.int16, #13
                    "Num_of_Delayed_Payment": np.float16, #14
                    "Changed_Credit_Limit": str,#np.float32, 15
                    "Num_Credit_Inquiries": np.int16, #16
                    "Credit_Mix": str, #17
                    "Outstanding_Debt": np.float32, #18
                    "Credit_Utilization_Ratio": np.float32, #19
                    "Credit_History_Age": str, #20
                    "Payment_of_Min_Amount": str, #21
                    "Total_EMI_per_month": np.float32, #22
                    "Amount_invested_monthly": np.float32, #23
                    "Payment_Behaviour": str, #24
                    "Monthly_Balance": np.float32, #25
                    "Credit_Score": str} #26 dtype=column_types, 
    
    ofJedi = pd.read_csv(filepath_or_buffer=path, on_bad_lines="skip")
    ofJedi.fillna(value=0, axis=0, inplace=True)

    ofJedi["Age"] = ofJedi["Age"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Annual_Income"] = ofJedi["Annual_Income"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Num_of_Loan"] = ofJedi["Num_of_Loan"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Num_of_Delayed_Payment"] = ofJedi["Num_of_Delayed_Payment"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Amount_invested_monthly"] = ofJedi["Amount_invested_monthly"].str.replace(pat=r"_*", repl="", regex=True)
    #ofJedi["Changed_Credit_Limit"] = ofJedi["Changed_Credit_Limit"].str.replace(pat=r"_*", repl="", regex=True)
    #ofJedi["Changed_Credit_Limit"] = ofJedi["Changed_Credit_Limit"].str.replace(pat="0-", repl="-")
    #ofJedi["Changed_Credit_Limit"] = ofJedi["Changed_Credit_Limit"].str.replace(pat="", repl="0")
    ofJedi["Monthly_Balance"] = ofJedi["Monthly_Balance"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Outstanding_Debt"] = ofJedi["Outstanding_Debt"].str.replace(pat=r"_*$", repl="", regex=True)

    ofJedi = ofJedi.astype(dtype=column_types)
    ofJedi.drop(columns=["ID", "Customer_ID", "Name", "SSN", "Month", "Occupation"], inplace=True)

    return ofJedi

In [ ]:
def count_null_rows(df):
    for col in df.columns:
        na_sum = df[col].isna().sum()
        sum = len(df)
        
        if na_sum > 0:
            print("{0}:\t{1}".format(col, (na_sum/sum)))

In [ ]:
def credit_history_age_clean(df):
    df.dropna(subset=["Credit_History_Age"], inplace=True)
    df["Credit_History_Age"] = df["Credit_History_Age"].str.slice(start=0, stop=2).astype(dtype=int)

In [ ]:
def check_objects_per(df: pd.DataFrame) -> None:
    count_rows = len(df)
    for col in df.columns:
        if df[col].dtype == object:
            print(col)
            print(df[col].value_counts()/count_rows)
            print("\n")

In [ ]:
def age_cleaner(df: pd.DataFrame, min_age: int, max_age: int) -> pd.DataFrame:
    return df.query("Age >= @min_age & Age <= @max_age")

In [ ]:
def age_analysis(x: pd.Series) -> None:
    print(x.head())
    print("Stardard Deviation:\t{0}\nMean:\t{1}\nVariance:\t{2}\nMaximum:\t{3}\nMinimum:\t{4}".format(x.std(), x.mean(), x.var(), x.max(), x.min()))

    print(x.describe())

    print("\nLength over 80:\t{0}".format(len(x[x > 80])))
    print("\nLength under 18:\t{0}".format(len(x[x < 18])))

    print("\nPercentage over 80:\t{0}".format(len(x[x > 80])/len(x)))
    print("\nPercentage under 18:\t{0}".format(len(x[x < 18])/len(x)))

    sns.boxplot(x=x)
    sns.displot(data=x, kind="kde")
    plt.show()

In [ ]:
def anual_income_analysis(x: pd.DataFrame) -> None:
    # Annual_Income", "Credit_Score"
    print(x["Annual_Income"].describe().apply(lambda i: format(i, "f")))

    sns.boxplot(x=x["Annual_Income"].apply(lambda i: format(i, "f")), y=x["Credit_Score"], native_scale=True, log_scale=False)
    sns.displot(data=x, kind="kde")
    plt.show()

In [ ]:
def generate_boxplot(df: pd.DataFrame) -> None:
    for col in df.columns:
        if df[col] != object:
            print(sns.boxplot(data=df[col]))

In [ ]:
credit_df = input_pipeline(path="train.csv")
credit_df.head()

In [ ]:
for col in credit_df.columns:
    print("{0}:\t{1}".format(col, credit_df[col].isna().sum()))

In [ ]:
len(credit_df)

In [ ]:
for col in credit_df.columns:
    na_sum = credit_df[col].isna().sum()
    sum = len(credit_df)

    if na_sum > 0 and credit_df[col].dtype==object:
        print("{0}:\t{1}".format(col, (na_sum/sum)))

In [ ]:
credit_df[credit_df["Type_of_Loan"].isna() & credit_df["Credit_History_Age"].isna()]

Type_of_Loan is too sparse, and too many nan values

In [ ]:
credit_df["Type_of_Loan"].value_counts()

In [ ]:
credit_df.drop(columns=["Type_of_Loan"], inplace=True)
credit_df

In [ ]:
count_null_rows(df=credit_df)

In [ ]:
credit_df["Credit_History_Age"].value_counts()

In [ ]:
credit_history_age_clean(credit_df)
credit_df

In [ ]:
count_null_rows(credit_df)

In [ ]:
credit_df.dropna(inplace=True)
credit_df

In [ ]:
for col in credit_df.columns:
    if credit_df[col].dtype == object:
        print(col)
        print(credit_df[col].value_counts())
        print("\n")

In [ ]:
credit_df[col].value_counts()

In [ ]:
credit_df["Credit_Score"].value_counts()/2

In [ ]:
check_objects_per(df=credit_df)

In [ ]:
credit_df.drop(index=credit_df[credit_df.Payment_Behaviour == "!@9#%8"].index, inplace=True)

In [ ]:
credit_df.drop(index=credit_df[credit_df.Credit_Mix == "_"].index, inplace=True)

In [ ]:
check_objects_per(df=credit_df)

In [ ]:
len(credit_df)

In [ ]:
credit_df.columns

In [ ]:
credit_df.reset_index(drop=True, inplace=True)
credit_df

Age analysis

In [ ]:
age_analysis(credit_df["Age"])

In [ ]:
credit_df.groupby(by=["Credit_Score"]).agg({"Age": "mean"})

In [ ]:
credit_df = age_cleaner(credit_df, 18, 80)
age_analysis(credit_df["Age"])

In [ ]:
credit_df.groupby(by=["Credit_Score"]).agg({"Age": "mean"})

### Annual_Income Analysis

In [ ]:
anual_income_analysis(x=credit_df.loc[:,["Annual_Income", "Credit_Score"]])